In [1]:
!pip install -q transformers datasets peft scikit-learn pandas pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.9 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigfram

In [2]:
from datasets import Dataset
import torch
from transformers import (
    BlipForQuestionAnswering,
    BlipProcessor,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model
from sklearn.model_selection import train_test_split
import pandas as pd
import os
from PIL import Image

# 1. Dataset Preparation -----------------------------------------------
# Load and flatten your dataset
df = pd.read_csv("/kaggle/input/merged-one/merged_vqa_dataset_output.csv")
BASE_IMAGE_DIR = "/kaggle/input/abo-dataset/images/small"

# Flatten into (image_path, question, answer) format
samples = []
for _, row in df.iterrows():
    for q_col, a_col in zip(['q1', 'q2', 'q3'], ['a1', 'a2', 'a3']):
        samples.append({
            "image_path": os.path.join(BASE_IMAGE_DIR, row['path']),
            "question": row[q_col],
            "answer": row[a_col]
        })

# Split into train/test
train_df, test_df = train_test_split(pd.DataFrame(samples), test_size=0.2, random_state=42)


2025-05-15 19:35:15.509927: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747337715.696420      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747337715.748965      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
test_df

,image_path,question,answer
9743,/kaggle/input/abo-dataset/images/small/28/2881...,What color is it?,Blue
31595,/kaggle/input/abo-dataset/images/small/30/3020...,What is the frame made of?,Wood
28281,/kaggle/input/abo-dataset/images/small/52/52e2...,What is in the image?,Handle
42139,/kaggle/input/abo-dataset/images/small/56/5678...,What color is it?,Gray
33152,/kaggle/input/abo-dataset/images/small/0f/0fd6...,How many screws are there?,Two
...,...,...,...
4523,/kaggle/input/abo-dataset/images/small/c0/c094...,What is the container made of?,Plastic
7231,/kaggle/input/abo-dataset/images/small/07/07c2...,What is the height?,6.0in
21016,/kaggle/input/abo-dataset/images/small/50/5019...,What color is the background?,blue
9242,/kaggle/input/abo-dataset/images/small/a2/a2bb...,How much is the height?,12in


In [4]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df, preserve_index=False)
test_dataset = Dataset.from_pandas(test_df, preserve_index=False)


In [5]:
wanted_cols = {"image_path", "question", "answer"}
for ds in [train_dataset, test_dataset]:
    extra_cols = [col for col in ds.column_names if col not in wanted_cols]
    if extra_cols:
        ds = ds.remove_columns(extra_cols)
wanted_cols = {"image_path", "question", "answer"}
for ds in [train_dataset, test_dataset]:
    extra_cols = [col for col in ds.column_names if col not in wanted_cols]
    if extra_cols:
        ds = ds.remove_columns(extra_cols)


In [6]:
from PIL import Image
from transformers import BlipProcessor

processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

def transform(batch):
    images = [Image.open(path).convert("RGB") for path in batch["image_path"]]
    encoding = processor(
        images,
        batch["question"],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=32
    )
    labels = processor.tokenizer(
        batch["answer"],
        padding="max_length",
        truncation=True,
        max_length=32,
        return_tensors="pt"
    ).input_ids

    batch["pixel_values"] = encoding["pixel_values"]
    batch["input_ids"] = encoding["input_ids"]
    batch["attention_mask"] = encoding["attention_mask"]
    batch["labels"] = labels
    return batch


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [7]:
train_dataset.set_transform(transform)
test_dataset.set_transform(transform)
train_dataset

Dataset({
    features: ['image_path', 'question', 'answer'],
    num_rows: 36883
})

In [8]:
from transformers import BlipForQuestionAnswering
from peft import LoraConfig, get_peft_model

model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["query", "key", "value", "fc1", "fc2"],  # Adjust if needed after inspecting model.named_modules()
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

trainable params: 2,506,752 || all params: 387,179,324 || trainable%: 0.6474


In [9]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./blip-lora-vqa",
    per_device_train_batch_size=4,   # Lower if you have OOM errors
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    fp16=True,
    learning_rate=1e-4,
    logging_steps=50,
    remove_unused_columns=False,
    report_to="none"
)


In [10]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    # tokenizer=processor,  # Optional, warning may appear but can be ignored for now
)


No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [11]:
train_results = trainer.train()
trainer.save_model("./blip-lora-vqa-final")
eval_results = trainer.evaluate()
print(f"Final evaluation results: {eval_results}")


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
50,9.600500
100,8.735600
150,8.427100
200,8.288700
250,8.227400
300,8.207700
350,8.180400
400,8.188900
450,8.174800
500,8.159900


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

Final evaluation results: {'eval_runtime': 667.1338, 'eval_samples_per_second': 13.822, 'eval_steps_per_second': 1.728, 'epoch': 3.0}


In [12]:
import os

model_dir = "./blip-lora-vqa-final"
print("Directory exists:", os.path.exists(model_dir))
print("Contents:", os.listdir(model_dir) if os.path.exists(model_dir) else "N/A")


Directory exists: True
Contents: ['adapter_model.safetensors', 'training_args.bin', 'README.md', 'adapter_config.json']


In [13]:
import torch
from transformers import BlipForQuestionAnswering, BlipProcessor
from PIL import Image

# Load model and processor
model_1 = BlipForQuestionAnswering.from_pretrained("./blip-lora-vqa-final")
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_1 = model_1.to(device)
model_1.eval()

# Preview predictions for the first 10 rows
for idx, row in test_df.head(10).iterrows():
    image = Image.open(row['image_path']).convert('RGB')
    question = row['question']
    true_answer = row['answer']

    # Preprocess
    inputs = processor(image, question, return_tensors="pt").to(device)

    # Generate prediction
    with torch.no_grad():
        out = model_1.generate(**inputs)
    predicted_answer = processor.decode(out[0], skip_special_tokens=True)

    print(f"Row {idx}:")
    print(f"  Question: {question}")
    print(f"  True Answer: {true_answer}")
    print(f"  Predicted Answer: {predicted_answer}")
    print("-" * 50)


Row 9743:
  Question: What color is it?
  True Answer: Blue
  Predicted Answer: blue
--------------------------------------------------
Row 31595:
  Question: What is the frame made of?
  True Answer: Wood
  Predicted Answer: wood
--------------------------------------------------
Row 28281:
  Question: What is in the image?
  True Answer: Handle
  Predicted Answer: handle
--------------------------------------------------
Row 42139:
  Question: What color is it?
  True Answer: Gray
  Predicted Answer: grey
--------------------------------------------------
Row 33152:
  Question: How many screws are there?
  True Answer: Two
  Predicted Answer: two
--------------------------------------------------
Row 28154:
  Question: What is the height?
  True Answer: 96cm
  Predicted Answer: 80cm
--------------------------------------------------
Row 39757:
  Question: What color is the case?
  True Answer: Pink
  Predicted Answer: pink
--------------------------------------------------
Row 19645:

In [14]:
import time
import torch
from transformers import BlipForQuestionAnswering, BlipProcessor
from PIL import Image

# Start timing
start_time = time.time()

# 1. Load your fine-tuned model and processor
model_1 = BlipForQuestionAnswering.from_pretrained("./blip-lora-vqa-final")
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

# 2. Move model to device (CPU or CUDA)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_1 = model_1.to(device)
model_1.eval()

# 3. Loop over your test DataFrame and generate answers
pred_answers = []

for idx, row in test_df.iterrows():
    image = Image.open(row['image_path']).convert('RGB')
    question = row['question']

    # Preprocess inputs
    inputs = processor(image, question, return_tensors="pt").to(device)

    # Generate answer
    with torch.no_grad():
        out = model_1.generate(**inputs)
    answer = processor.decode(out[0], skip_special_tokens=True)
    pred_answers.append(answer)

# 4. Add predictions to your DataFrame as a new column
test_df['predicted_answer'] = pred_answers

# End timing
end_time = time.time()
elapsed_time = end_time - start_time

# 5. Display or save results
print(test_df[['image_path', 'question', 'answer', 'predicted_answer']].head())
print(f"\n⏱️ Total time taken: {elapsed_time:.2f} seconds")


                                              image_path  \
9743   /kaggle/input/abo-dataset/images/small/28/2881...   
31595  /kaggle/input/abo-dataset/images/small/30/3020...   
28281  /kaggle/input/abo-dataset/images/small/52/52e2...   
42139  /kaggle/input/abo-dataset/images/small/56/5678...   
33152  /kaggle/input/abo-dataset/images/small/0f/0fd6...   

                         question  answer predicted_answer  
9743            What color is it?    Blue             blue  
31595  What is the frame made of?    Wood             wood  
28281       What is in the image?  Handle           handle  
42139           What color is it?    Gray             grey  
33152  How many screws are there?     Two              two  

⏱️ Total time taken: 1085.49 seconds


In [15]:
# Assuming 'test_df' already has columns: 'answer' (ground truth) and 'predicted_answer'
# We'll add 'is_true' column: 1 if prediction matches ground truth, else 0

# Normalize answers for comparison (optional but recommended)
def normalize(text):
    return str(text).strip().lower()

test_df['is_true'] = (test_df['predicted_answer'].apply(normalize) == test_df['answer'].apply(normalize)).astype(int)

# Count number of true and false predictions
num_true = test_df['is_true'].sum()
num_false = len(test_df) - num_true

print(f"Number of correct predictions (is_true=1): {num_true}")
print(f"Number of incorrect predictions (is_true=0): {num_false}")

# Save to CSV
test_df.to_csv('LORS_PRED_8.csv', index=False)
print("Results saved to LORS_PRED_8.csv")


Number of correct predictions (is_true=1): 6088
Number of incorrect predictions (is_true=0): 3133
Results saved to LORS_PRED_8.csv


In [16]:
pip install bitsandbytes transformers


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.9 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [17]:
import os
from transformers import BlipForQuestionAnswering, BitsAndBytesConfig

model_dir = "/kaggle/working/blip-lora-vqa-final"
print("Model directory contents:", os.listdir(model_dir))  # Debug: See what files are present

# bnb_config = BitsAndBytesConfig(load_in_8bit=True, llm_int8_threshold=6.0)

# model_2 = BlipForQuestionAnswering.from_pretrained(
#     model_dir,
#     quantization_config=bnb_config,
#     device_map="auto",
#     local_files_only=True
# )


Model directory contents: ['adapter_model.safetensors', 'training_args.bin', 'README.md', 'adapter_config.json']


In [23]:
pip install -U transformers accelerate bitsandbytes


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 6.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.5.2
    Uninstalling accelerate-1.5.2:
      Successfully uninstalled accelerate-1.5.2
Note: you may need to restart the kernel to use updated packages.


Version 1: Mixed Precision + Manual .to(device)

In [29]:
import os
import time
import torch
from transformers import BlipForQuestionAnswering, BlipProcessor
from PIL import Image

# Start timer
start_time = time.time()

# Load processor and model
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model_dir = os.path.abspath("./blip-lora-vqa-final")
print("Loading model from:", model_dir)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_2 = BlipForQuestionAnswering.from_pretrained(
    model_dir,
    local_files_only=True
).to(device)
model_2.eval()

# Inference loop
pred_answers = []
for idx, row in test_df.iterrows():
    image = Image.open(row['image_path']).convert('RGB')
    question = row['question']
    inputs = processor(image, question, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad(), torch.autocast(device_type=device.type, dtype=torch.bfloat16):
        out = model_2.generate(**inputs)
    
    answer = processor.decode(out[0], skip_special_tokens=True)
    pred_answers.append(answer)

# Save and display results
test_df['predicted_answer'] = pred_answers
end_time = time.time()

print(test_df[['image_path', 'question', 'answer', 'predicted_answer']].head())
print(f"\n⏱️ Total time taken (Mixed Precision): {end_time - start_time:.2f} seconds")


Loading model from: /kaggle/working/blip-lora-vqa-final
                                              image_path  \
9743   /kaggle/input/abo-dataset/images/small/28/2881...   
31595  /kaggle/input/abo-dataset/images/small/30/3020...   
28281  /kaggle/input/abo-dataset/images/small/52/52e2...   
42139  /kaggle/input/abo-dataset/images/small/56/5678...   
33152  /kaggle/input/abo-dataset/images/small/0f/0fd6...   

                         question  answer predicted_answer  
9743            What color is it?    Blue             blue  
31595  What is the frame made of?    Wood             wood  
28281       What is in the image?  Handle           handle  
42139           What color is it?    Gray             grey  
33152  How many screws are there?     Two              two  

⏱️ Total time taken (Mixed Precision): 1649.71 seconds


Version 2: Batched Inference + Mixed Precision

In [26]:
import os
import time
import torch
from transformers import BlipForQuestionAnswering, BlipProcessor
from PIL import Image

# Start timer
start_time = time.time()

# Load processor and model
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model_dir = os.path.abspath("./blip-lora-vqa-final")
print("Loading model from:", model_dir)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_2 = BlipForQuestionAnswering.from_pretrained(
    model_dir,
    local_files_only=True
).to(device)
model_2.eval()

# Batch inference
batch_size = 4
pred_answers = []

for i in range(0, len(test_df), batch_size):
    batch = test_df.iloc[i:i+batch_size]
    images = [Image.open(p).convert("RGB") for p in batch['image_path']]
    questions = list(batch['question'])

    inputs = processor(images, questions, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad(), torch.autocast(device_type=device.type, dtype=torch.bfloat16):
        out = model_2.generate(**inputs)

    answers = processor.batch_decode(out, skip_special_tokens=True)
    pred_answers.extend(answers)

# Save and display results
test_df['predicted_answer'] = pred_answers
end_time = time.time()

print(test_df[['image_path', 'question', 'answer', 'predicted_answer']].head())
print(f"\n⏱️ Total time taken (Batched + Mixed Precision): {end_time - start_time:.2f} seconds")


Loading model from: /kaggle/working/blip-lora-vqa-final
                                              image_path  \
9743   /kaggle/input/abo-dataset/images/small/28/2881...   
31595  /kaggle/input/abo-dataset/images/small/30/3020...   
28281  /kaggle/input/abo-dataset/images/small/52/52e2...   
42139  /kaggle/input/abo-dataset/images/small/56/5678...   
33152  /kaggle/input/abo-dataset/images/small/0f/0fd6...   

                         question  answer predicted_answer  
9743            What color is it?    Blue             blue  
31595  What is the frame made of?    Wood             wood  
28281       What is in the image?  Handle           handle  
42139           What color is it?    Gray             grey  
33152  How many screws are there?     Two              two  

⏱️ Total time taken (Batched + Mixed Precision): 1056.12 seconds


In [28]:
# import os
# import time
# import torch
# from transformers import BlipForQuestionAnswering, BlipProcessor, BitsAndBytesConfig
# from PIL import Image

# # Start timing
# start_time = time.time()

# # Define model path
# model_dir = os.path.abspath("./blip-lora-vqa-final")  # change if needed
# print("Loading model from:", model_dir)

# # Load processor
# processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

# # Quantization config
# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True,              # 8-bit quantization
#     llm_int8_threshold=6.0,
#     llm_int8_skip_modules=None,
# )

# # Load quantized model
# model_2 = BlipForQuestionAnswering.from_pretrained(
#     model_dir,
#     device_map="auto",
#     quantization_config=bnb_config,
#     local_files_only=True
# )
# model_2.eval()

# # Move model to appropriate device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Inference loop
# pred_answers = []
# for idx, row in test_df.iterrows():
#     image = Image.open(row['image_path']).convert('RGB')
#     question = row['question']
#     inputs = processor(image, question, return_tensors="pt").to(device)
#     with torch.no_grad():
#         out = model_2.generate(**inputs)
#     answer = processor.decode(out[0], skip_special_tokens=True)
#     pred_answers.append(answer)

# test_df['predicted_answer'] = pred_answers

# # End timing
# end_time = time.time()
# print(test_df[['image_path', 'question', 'answer', 'predicted_answer']].head())
# print(f"\n⏱️ Total time taken: {end_time - start_time:.2f} seconds")


In [27]:
def normalize(text):
    return str(text).strip().lower()

test_df['is_true'] = (test_df['predicted_answer'].apply(normalize) == test_df['answer'].apply(normalize)).astype(int)
num_true = test_df['is_true'].sum()
num_false = len(test_df) - num_true

print(f"Number of correct predictions (is_true=1): {num_true}")
print(f"Number of incorrect predictions (is_true=0): {num_false}")
test_df.to_csv('LORS_PRED_8_QUANT.csv', index=False)
print("Results saved to LORS_PRED_8_QUANT.csv")


Number of correct predictions (is_true=1): 6059
Number of incorrect predictions (is_true=0): 3162
Results saved to LORS_PRED_8_QUANT.csv


Flash Attention (with xformers backend)

In [30]:
pip install xformers


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 58.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 5.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 73.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 1.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 6.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 10.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━

In [34]:
import os
import time
import torch
import pandas as pd
from transformers import BlipForQuestionAnswering, BlipProcessor
from PIL import Image

# Your test dataframe (replace with actual loading logic)
# Example:
# test_df = pd.read_csv("test_data.csv")  # Columns: image_path, question, answer

# Setup
torch.backends.cuda.matmul.allow_tf32 = True
os.environ["XFORMERS_FORCE_DISABLE"] = "0"

start_time = time.time()

# Load processor and model
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load fine-tuned model (no device_map, load directly to single GPU)
model_dir = os.path.abspath("./blip-lora-vqa-final")
print("Loading model from:", model_dir)
model_2 = BlipForQuestionAnswering.from_pretrained(
    model_dir,
    local_files_only=True,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)
model_2.to(device)
model_2.eval()

# Inference loop
pred_answers = []

for idx, row in test_df.iterrows():
    image = Image.open(row['image_path']).convert('RGB')
    question = row['question']

    # Prepare inputs
    inputs = processor(image, question, return_tensors="pt")
    
    # Move pixel_values to float16, others remain as int
    inputs['pixel_values'] = inputs['pixel_values'].to(device=device, dtype=torch.float16)
    inputs['input_ids'] = inputs['input_ids'].to(device)
    inputs['attention_mask'] = inputs['attention_mask'].to(device)

    with torch.no_grad():
        out = model_2.generate(**inputs)

    answer = processor.decode(out[0], skip_special_tokens=True)
    pred_answers.append(answer)

# Save predictions to DataFrame
test_df['predicted_answer'] = pred_answers

end_time = time.time()
print(test_df[['image_path', 'question', 'answer', 'predicted_answer']].head())
print(f"\n⏱️ Flash-Attn Optimized Time: {end_time - start_time:.2f} seconds")


Using device: cuda:0
Loading model from: /kaggle/working/blip-lora-vqa-final
                                              image_path  \
9743   /kaggle/input/abo-dataset/images/small/28/2881...   
31595  /kaggle/input/abo-dataset/images/small/30/3020...   
28281  /kaggle/input/abo-dataset/images/small/52/52e2...   
42139  /kaggle/input/abo-dataset/images/small/56/5678...   
33152  /kaggle/input/abo-dataset/images/small/0f/0fd6...   

                         question  answer predicted_answer  
9743            What color is it?    Blue             blue  
31595  What is the frame made of?    Wood             wood  
28281       What is in the image?  Handle           handle  
42139           What color is it?    Gray             grey  
33152  How many screws are there?     Two              two  

⏱️ Flash-Attn Optimized Time: 1038.51 seconds


2. TorchScript (Static Graph Compilation)

In [36]:
import os
import time
import torch
from transformers import BlipForQuestionAnswering, BlipProcessor
from PIL import Image

start_time = time.time()

processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model_dir = os.path.abspath("./blip-lora-vqa-final")
print("Loading model from:", model_dir)

model_2 = BlipForQuestionAnswering.from_pretrained(
    model_dir,
    local_files_only=True
).to("cuda").eval()

pred_answers = []
for idx, row in test_df.iterrows():
    image = Image.open(row['image_path']).convert('RGB')
    question = row['question']
    inputs = processor(image, question, return_tensors="pt").to("cuda")
    with torch.no_grad():
        out = model_2.generate(**inputs)
    answer = processor.decode(out[0], skip_special_tokens=True)
    pred_answers.append(answer)

test_df['predicted_answer'] = pred_answers
end_time = time.time()

print(test_df[['image_path', 'question', 'answer', 'predicted_answer']].head())
print(f"\n⏱️ Inference Time: {end_time - start_time:.2f} seconds")


Loading model from: /kaggle/working/blip-lora-vqa-final
                                              image_path  \
9743   /kaggle/input/abo-dataset/images/small/28/2881...   
31595  /kaggle/input/abo-dataset/images/small/30/3020...   
28281  /kaggle/input/abo-dataset/images/small/52/52e2...   
42139  /kaggle/input/abo-dataset/images/small/56/5678...   
33152  /kaggle/input/abo-dataset/images/small/0f/0fd6...   

                         question  answer predicted_answer  
9743            What color is it?    Blue             blue  
31595  What is the frame made of?    Wood             wood  
28281       What is in the image?  Handle           handle  
42139           What color is it?    Gray             grey  
33152  How many screws are there?     Two              two  

⏱️ Inference Time: 1104.92 seconds
